In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

#  FROM : https://www.tensorflow.org/tutorials/layers#building_the_cnn_mnist_classifier
#  CODE : https://www.tensorflow.org/code/tensorflow/examples/tutorials/layers/cnn_mnist.py

"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
print('Tensorflow:',tf.__version__)
import sys
print(sys.version)

In [35]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)
  #logits = tf.layers.dense(inputs=dense, units=10)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        #optimizer="SGD")
        optimizer="Adam")

  # Generate Predictions
  predictions = {
      "classes":       tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor"), 
      "logits":        logits,
      "gradient":      tf.gradients(loss, input_layer),
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [36]:
# Load training and eval data
mnist = learn.datasets.load_dataset("mnist")

train_data   = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

eval_data    = mnist.test.images  # Returns np.array
eval_labels  = np.asarray(mnist.test.labels, dtype=np.int32)

eval_labels[7]

In [37]:
# Create the Estimator : https://www.tensorflow.org/extend/estimators
import os
this_dir = os.getcwd()

mnist_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="mnist_convnet_model")  # This is relative to the ipynb
      #model_fn=cnn_model_fn, model_dir=os.path.join(this_dir, "mnist_convnet_model"))  # This is system-absolute

# NB :  Check the checkpoints file in 'mnist_convnet_model has filenames that are in same directory

In [38]:
mnist_classifier.get_variable_names()
#mnist_classifier.get_variable_value('conv2d/bias')

In [ ]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook( tensors=tensors_to_log, every_n_iter=50 )

# Train the model
mnist_classifier.fit(
  x=train_data,
  y=train_labels,
  batch_size=100,
  steps=50000/100 * 5,
#  steps=20000,
#  monitors=[logging_hook]
)

In [ ]:
#mnist_classifier.save()

In [ ]:
# Configure the accuracy metric for evaluation
cnn_metrics = {
  "accuracy":
      learn.MetricSpec(
          metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# Evaluate the model and print results
cnn_eval_results = mnist_classifier.evaluate( x=eval_data, y=eval_labels, metrics=cnn_metrics)

print(cnn_eval_results)

Ok, so the built Estimator gets 99.26% accuracy on the test set in <20 secs on CPU.

### Next Steps

Let's :

*  go through the training set and store the logits for [the valid?] ALL the numbers;

*  build an AutoEncoder on the logits, which minimises reconstruction error;

*  histogram the reconstruction error to find a bound above which we can reject an input image;

*  create adversarial examples that fool the original Estimator;

*  check that the adversarial examples score outside the bound;

*  attempt to create adversarial examples on an updated network that includes the autoencoder bound as a gating function on the rest of the outputs;

*  create an infoGAN network for MNIST that allows us to create digits that are 'between' two classes;

*  score the reconstruction error of the between images to look at the rejection regions (which hopefully isolate the islands of acceptance from one another)




#### Get logit representation for all training examples


In [6]:
# Evaluate the model and gather the results
predictions = mnist_classifier.predict( x=train_data, batch_size=100, as_iterable=True)

#for p in predictions:
#    print(p)
#    break

train_data_logits = np.array( [ p['logits'] for p in predictions ])

predictions = mnist_classifier.predict( x=eval_data, batch_size=100, as_iterable=True)
eval_data_logits  = np.array( [ p['logits'] for p in predictions ])

train_data_logits.shape, eval_data_logits.shape

In [16]:
# Predictions take ~ 60secs

# So, let's build an autoencoder 'regression' model with a hidden layer with input/2 units 

def autoencoder_model_fn(features, labels, mode):
  #print("features.shape", features.get_shape())
  #dim = features.get_shape()[1]
  dim = 10

  # Input Layer
  input_layer = tf.reshape(features, [-1, dim])

  # Encoder Dense Layer
  # Densely connected layer with size/2 neurons
  # Input Tensor Shape: [batch_size, 10]
  # Output Tensor Shape: [batch_size, 10/2]
  #dense = tf.layers.dense(inputs=input_layer, units=dim, activation=tf.nn.relu)
  dense = tf.layers.dense(inputs=input_layer, units=dim, activation=tf.nn.elu)  # ELU!

  # Add dropout operation; 0.6 probability that element will be kept
  #dropout = tf.layers.dropout(
  #    inputs=dense, rate=0.9, training=mode == learn.ModeKeys.TRAIN)

  # Decoder Dense Layer
  # Input Tensor Shape: [batch_size, 10/2]
  # Output Tensor Shape: [batch_size, 10]
  #output_layer = tf.layers.dense(inputs=dropout, units=dim)
  output_layer = tf.layers.dense(inputs=dense, units=dim)  # Linear activation

  loss = None
  train_op = None

  ## Calculate Loss (for both TRAIN and EVAL modes)
  #if mode != learn.ModeKeys.INFER:
  #  loss = tf.losses.mean_squared_error( input_layer, output_layer )

  loss = tf.losses.mean_squared_error( output_layer, labels )

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="Adam")

  # Generate Predictions
  predictions = {
      "rmse": loss,
      "regenerated":output_layer, 
      "gradient": tf.gradients(loss, input_layer),
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [17]:
mnist_autoencoder = learn.Estimator(
      model_fn=autoencoder_model_fn, model_dir="mnist_model/autoencoder")

In [9]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
#tensors_to_log = {"probabilities": "softmax_tensor"}
#logging_hook = tf.train.LoggingTensorHook( tensors=tensors_to_log, every_n_iter=50 )

# Train the model
mnist_autoencoder.fit(
  x=train_data_logits,
  y=train_data_logits,
  batch_size=100,
  steps=50000/100 * 10,
#  steps=20000,
#  monitors=[logging_hook]
)

*  n/2 hidden INFO:tensorflow:Saving checkpoints for 25000 into mnist_model/autoencoder/model.ckpt.
*  n/2 hidden INFO:tensorflow:Loss for final step: 1.2686.

*  2xReLU INFO:tensorflow:Saving checkpoints for 25000 into mnist_model/autoencoder/model.ckpt.
*  2xReLU INFO:tensorflow:Loss for final step: 1.47784e-05.

*  ELU+ReLU INFO:tensorflow:Saving checkpoints for 5000 into mnist_model/autoencoder/model.ckpt.
*  ELU+ReLU INFO:tensorflow:Loss for final step: 0.00331942.

In [10]:
# Configure the accuracy metric for evaluation
auto_metrics = {
  "loss":learn.MetricSpec(metric_fn=tf.metrics.mean_squared_error, prediction_key="regenerated"),
}

# Evaluate the model and print results
autoencoder_eval_results = mnist_autoencoder.evaluate( x=eval_data_logits, y=eval_data_logits, metrics=auto_metrics)

print(autoencoder_eval_results)

*   {'loss': 2.3425782e-05, 'global_step': 13001} => This autoenoder thing works

Let's now create (some) adversarial digits for MNIST


In [11]:
train_offset = 17
# train_data[train_offset]  # This is a flat numpy array with an image in it
#train_labels[train_offset]  # This the digit label for that image

image_orig = train_data[train_offset]

In [14]:
#class_predictions = mnist_classifier.predict( x=np.array([image_orig]), batch_size=1, as_iterable=False)
class_predictions = mnist_classifier.predict( x=image_orig, as_iterable=False)
class_predictions['probabilities'][0]

#class_predictions = mnist_classifier.predict( x=image_orig, outputs=['probabilities'], as_iterable=False)
#class_predictions

### Intuition behind 'gradient' for explicit inception version ...

In [ ]:
## Set the graph for the Inception model as the default graph,
## so that all changes inside this with-block are done to that graph.
#with model.graph.as_default():
#    # Add a placeholder variable for the target class-number.
#    # This will be set to e.g. 300 for the 'bookcase' class.
#    pl_cls_target = tf.placeholder(dtype=tf.int32)
#
#    # Add a new loss-function. This is the cross-entropy.
#    # See Tutorial #01 for an explanation of cross-entropy.
#    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_logits, labels=[pl_cls_target])
#
#    # Get the gradient for the loss-function with regard to
#    # the resized input image.
#    gradient = tf.gradients(loss, resized_image)

In [39]:
# Configure the accuracy metric for evaluation
def metric_first_value(predictions, labels):
    return predictions[0]

model_gradient = {
#  "gradient":learn.MetricSpec(metric_fn=tf.metrics.mean, weight_key=None, prediction_key="gradient"),
  "gradient":learn.MetricSpec(metric_fn=metric_first_value, prediction_key="gradient"),
}

# Evaluate the model and print results
cnn_gradient = mnist_classifier.evaluate( x=np.array([image_orig]), y=np.array([7], dtype='int'), metrics=model_gradient)
#cnn_gradient = mnist_classifier.evaluate( x=image_orig, y=np.int32(7), metrics=model_gradient)

cnn_gradient.shape

In [ ]:
def find_adversary_noise(image_np, cls_target, model, noise_limit=3.0, required_score=0.99, max_iterations=50):
    """
    Find the noise that must be added to the given image so
    that it is classified as the target-class by the given model.
    
    image_np: numpy image in correct format
    cls_target: Target class-number (integer between 0-n_classes).
    noise_limit: Limit for pixel-values in the noise.
    required_score: Stop when target-class score reaches this.
    max_iterations: Max number of optimization iterations to perform.
    """

    # Create a feed-dict with the image.
    #feed_dict = model._create_feed_dict(image_path=image_path)

    # Use TensorFlow to calculate the predicted class-scores
    # (aka. probabilities) as well as the resized image.
    #pred, image = session.run([y_pred, resized_image], feed_dict=feed_dict)
    pred_all = model.predict( x=image_np, as_iterable=False)

    # Convert to one-dimensional array.
    #pred = np.squeeze(pred)
    pred = pred_all['probabilities'][0]

    # Predicted class-number.
    cls_source = np.argmax(pred)

    # Score for the predicted class (aka. probability or confidence).
    score_source_org = pred.max()

    # Names for the source and target classes.
    #name_source = model.name_lookup.cls_to_name(cls_source, only_first_name=True)
    #name_target = model.name_lookup.cls_to_name(cls_target, only_first_name=True)

    # Initialize the noise to zero.
    noise = 0.

    # Perform a number of optimization iterations to find
    # the noise that causes mis-classification of the input image.
    for i in range(max_iterations):
        print("Iteration:", i)

        # The noisy image is just the sum of the input image and noise.
        noisy_image = image + noise
        
        #print(noisy_image.shape)
        #save image
        #scipy.misc.imsave('./images/noisy-processing.jpg', noisy_image)
        #test_noisy_image = noisy_image

        # Ensure the pixel-values of the noisy image are between
        # 0 and 255 like a real image. If we allowed pixel-values
        # outside this range then maybe the mis-classification would
        # be due to this 'illegal' input breaking the Inception model.
        noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)
        
        #test_noisy_image = noisy_image
        #print(noisy_image)

        # Create a feed-dict. This feeds the noisy image to the
        # tensor in the graph that holds the resized image, because
        # this is the final stage for inputting raw image data.
        # This also feeds the target class-number that we desire.
        feed_dict = {model.tensor_name_resized_image: noisy_image, pl_cls_target: cls_target}

        # Calculate the predicted class-scores as well as the gradient.
        pred, grad = session.run([y_pred, gradient], feed_dict=feed_dict)
        
        pred_true = pred
        print(pred_true[0][409])
        #for i in range(0,1000):
        #    if pred_true[0][i] > 0.005:
        #        print('close to class '+str(i)+' which is' + model.name_lookup.cls_to_name(i, only_first_name=True))
              
        #scipy.misc.imsave('temp-image.jpg', noisy_image[0])
        new_noisy_image = cv2.cvtColor(noisy_image[0], cv2.COLOR_BGR2RGB)
        cv2.imwrite('./images/noisy_image-temp.jpg',new_noisy_image)

        pred2 = model.classify('./images/noisy_image-temp.jpg')
        model.print_scores(pred=pred2, k=10, only_first_name=True) 

        # Convert the predicted class-scores to a one-dim array.
        pred = np.squeeze(pred)

        # The scores (probabilities) for the source and target classes.
        score_source = pred[cls_source]
        score_target = pred[cls_target]

        # Squeeze the dimensionality for the gradient-array.
        grad = np.array(grad).squeeze()

        # The gradient now tells us how much we need to change the
        # noisy input image in order to move the predicted class
        # closer to the desired target-class.

        # Calculate the max of the absolute gradient values.
        # This is used to calculate the step-size.
        grad_absmax = np.abs(grad).max()
        
        # If the gradient is very small then use a lower limit,
        # because we will use it as a divisor.
        if grad_absmax < 1e-10:
            grad_absmax = 1e-10

        # Calculate the step-size for updating the image-noise.
        # This ensures that at least one pixel colour is changed by 7.
        # Recall that pixel colours can have 255 different values.
        # This step-size was found to give fast convergence.
        step_size = 7 / grad_absmax

        # Print the score etc. for the source-class.
        msg = "Source score: {0:>7.2%}, class-number: {1:>4}, class-name: {2}"
        print(msg.format(score_source, cls_source, name_source))

        # Print the score etc. for the target-class.
        msg = "Target score: {0:>7.2%}, class-number: {1:>4}, class-name: {2}"
        print(msg.format(score_target, cls_target, name_target))

        # Print statistics for the gradient.
        msg = "Gradient min: {0:>9.6f}, max: {1:>9.6f}, stepsize: {2:>9.2f}"
        print(msg.format(grad.min(), grad.max(), step_size))
        
        #for i in pred_true:
        #    if pred_true[i] > 0.9:
        #        print(i + ' = ' + "hi")
        #print(pred_true[0])
        
        #save image
        #new_noisy_image = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB)
        #new_normal_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #cv2.imwrite('./images/noisy_image-processing.jpg',new_noisy_image)
        #cv2.imwrite('./images/normal_image.jpg',new_normal_image)

        # Newline.
        print()
        noisy_image = np.clip(a=noisy_image, a_min=0.0, a_max=255.0)
        
        
        # If the score for the target-class is not high enough.
        if score_target < required_score:
            # Update the image-noise by subtracting the gradient
            # scaled by the step-size.
            noise -= step_size * grad

            # Ensure the noise is within the desired range.
            # This avoids distorting the image too much.
            noise = np.clip(a=noise,
                            a_min=-noise_limit,
                            a_max=noise_limit)
        else:
            # Abort the optimization because the score is high enough.
            break

    return image.squeeze(), noisy_image.squeeze(), noise, \
           name_source, name_target, \
           score_source, score_source_org, score_target

In [ ]:
# tf.getDefaultGraph().finalize()